# Chapter 9: Bench Time: Simple Power Analysis

This notebook is a companion to Chapter 9 of The Hardware Hacking Handbook by Jasper van Woudenberg and Colin O'Flynn. In contrast to other chapter notebooks, this lab is standalone and does not correspond 1:1 to sections.

© 2021. This work is licensed under a [CC BY-SA 4.0 license](https://creativecommons.org/licenses/by-sa/4.0/). 

**SUMMARY:** *This tutorial will introduce you to breaking devices by determining when a device is performing certain operations. Our target device will be performing a simple password check, and we will demonstrate how to perform a basic power analysis.*

**LEARNING OUTCOMES:**

* How power can be used to determine timing information.
* Plotting multiple iterations while varying input data to find interesting locations.
* Using difference of waveforms to find interesting locations.
* Performing power captures with ChipWhisperer hardware (hardware only)


## Prerequisites

Hold up! Before you continue, check you've done the following tutorials:

* ☑ Jupyter Notebook Intro (you should be OK with plotting & running blocks).
* ☑ SCA101 Intro (you should have an idea of how to get hardware-specific versions running).

## Power Trace Gathering

At this point you've got to insert code to perform the power trace capture. There are two options here:
* Capture from physical device.
* Read from a file.

The following will capture data from the ChipWhisperer-Nano - you can adjust this to work with the ChipWhisperer-Lite as well. There is also a 'simulated' version including pre-recorded power traces if you don't have hardware. See the next section if that describes you - don't run these cells if you don't have the hardware attached.

In [ ]:
#VERSION = 'HARDWARE'
VERSION = 'SIMULATED'

### Power Trace Gathering - with Hardware

In [ ]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWNANO'
SS_VER = 'SS_VER_1_1'

In [ ]:
if VERSION == 'HARDWARE':
    %run "Helper_Scripts/Setup_Generic.ipynb"    

In [ ]:
%%bash -s "$PLATFORM" "$SS_VER"
cd ../hardware/victims/firmware/basic-passwdcheck
make PLATFORM=$1 CRYPTO_TARGET=NONE SS_VER=$2

In [ ]:
if VERSION == 'HARDWARE':
    cw.program_target(scope, prog, "../hardware/victims/firmware/basic-passwdcheck/basic-passwdcheck-{}.hex".format(PLATFORM))

In [ ]:
def cap_pass_trace(pass_guess):
    reset_target(scope)
    num_char = target.in_waiting()
    while num_char > 0:
        target.read(num_char, 10)
        time.sleep(0.01)
        num_char = target.in_waiting()

    scope.arm()
    target.write(pass_guess)
    ret = scope.capture()
    if ret:
        print('Timeout happened during acquisition')

    trace = scope.get_last_trace()
    return trace

In [ ]:
if VERSION == 'HARDWARE':
    scope.adc.samples = 3000
    trace_test = cap_pass_trace("h\n")

    #Basic sanity check
    assert(len(trace_test) == 3000)
    print("✔️ OK to continue (HARDWARE)!")

### Power Trace Gathering - without Hardware (SIMULATED)

If you don't have power traces, we can also simulate the attack by using pre-recorded power traces. Be warned of one major difference: you only have a limited set of data to pull from. This means you CANNOT try totally arbitrary "guesses", however if you are just running the examples in this notebook it will work.

If you *really* want to experiment you will need the hardware in front of you to have it respond to any possible input.

In [ ]:
if VERSION == 'SIMULATED':
    
    import pickle
    import random
    import time

    traces_to_load = pickle.load(open(r"traces/lab2_1b_passwords_full.p", "rb"))

    def cap_pass_trace(pass_guess):
        if pass_guess.endswith("\n") is False:
            raise ValueError("Password guess must end with \\n")

        pass_guess = pass_guess.strip("\n")

        known_passwd = "h0px3"

        trylist = "abcdefghijklmnopqrstuvwxyz0123456789 \x00"

        if len(pass_guess) > 5:
            raise ValueError("Only guesses up to 5 chars recorded, sorry about that.")

        for a in pass_guess:
            if a not in trylist:
                raise ValueError("Part of guess (%c) not in recorded enumeration list (%s)"%(a, trylist))

        #Only recorded is correct passwords
        recorded_pw = ""
        for i in range(0, len(pass_guess)):
            if known_passwd[i] != pass_guess[i]:
                recorded_pw += " "
            else:
                recorded_pw += pass_guess[i]

        time.sleep(0.05)

        return traces_to_load[recorded_pw][random.randint(0, 99)]

    trace_test = cap_pass_trace("h\n")

    #Basic sanity check
    assert(len(trace_test) == 3000)
    print("✔️ OK to continue (SIMULATED)!")

## Exploration

So what can we do with this? While first off - I'm going to cheat, and tell you that we have a preset password that starts with `h`, and it's 5 characters long. But that's the only hint so far - what can you do? While first off, let's try plotting a comparison of `h` to something else.

If you need a reminder of how to do a plot - see the matplotlib section of the **Jupyter Introduction** notebook.

The following cell shows you how to capture one power trace with `h` sent as a password. From there:

1. Try adding the plotting code and see what it looks like.
2. Send different passwords to the device. We're only going to look at the difference between a password starting with `h` and something else right now.
3. Plot the different waveforms.

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
plt.figure()
plt.plot(cap_pass_trace("h\n"))
plt.plot(cap_pass_trace("0\n"))
plt.show()

For reference, the output should look something like this - but be aware it will change with compiler output! So this is only a vague example, your output may look rather difference.
![](spa_password_h_vs_0_overview.png)
If you are using the `%matplotlib notebook` magic, you can zoom in at the start. What you want to notice is there is two code paths taken, depending on a correct or incorrect path. Here for example is a correct & incorrect character processed:
![](spa_password_h_vs_0_zoomed.png)

OK interesting -- what's next? Let's plot every possible password character we could send.

Our password implementation only recognizes characters in the list `abcdefghijklmnopqrstuvwxyz0123456789`, so we're going to limit it to those valid characters for now.

Write some code in the following block that implements the following algorithm:

    for CHARACTER in LIST_OF_VALID_CHARACTERS:
        trace = cap_pass_trace(CHARACTER + "\n")
        plot(trace)
        
The above isn't quite valid code - so massage it into place! You also may notice the traces are way too long - you might want to make a more narrow plot that only does the first say 500 samples of the power trace.

---
📝**Plotting Note**

If using `matplotlib` for plotting, you might need to add a `plt.figure()` at the start to make a new figure. Otherwise you might find your plot ends up in the figure above! If you don't see the plots, sometimes a `plt.show()` is needed at the end.

---

In [ ]:
%matplotlib notebook
from tqdm import tqdm
plt.figure()
for c in tqdm('abcdefghijklmnopqrstuvwxyz0123456789'):
    trace = cap_pass_trace(c + "\n")
    plt.plot(trace[0:500])

The end result should be if you zoom in, you'll see there is a location where a single "outlier" trace doesn't follow the path of all the other traces. That is great news, since it means we learn something about the system from power analysis.

![](spa_password_list_char1.png)
Using your loop - you can also try modifying the analysis to capture a correct "first" character, and then every other wrong second character. Do you see a difference you might be able to detect?

The pseudo-code would look something like this:

    for CHARACTER in LIST_OF_VALID_CHARACTERS:
        trace = cap_pass_trace("h" + CHARACTER + "\n")
        plot(trace)

Give that a shot in your earlier code-block, and then let's try and automate this attack to understand the data a little better.

## Automating an Attack against One Character

To start with - we're going to automate an attack against a **single** character of the password. Since we don't know the password (let's assume), we'll use a strategy of comparing all possible inputs together.

An easy way to do this might be to use something that we know can't be part of the valid password. As long as it's processed the same way, this will work just fine. So for now, let's use a password as `0x00` (i.e., a null byte). We can compare the null byte to processing something else:

In [ ]:
%matplotlib notebook
import matplotlib.pylab as plt

plt.figure()
ref_trace = cap_pass_trace("\x00\n")[0:500]
plt.plot(ref_trace)
other_trace = cap_pass_trace("c\n")[0:500]
plt.plot(other_trace)
plt.show()

This will plot a trace with an input of "\x00" - a null password! This is an invalid character, and seems to be processed as any other invalid password.

Let's make this a little more obvious, and plot the difference between a known reference & every other capture. You need to write some code that does something like this:

    ref_trace = cap_pass_trace( "\x00\n")

    for CHARACTER in LIST_OF_VALID_CHARACTERS:
        trace = cap_pass_trace(CHARACTER + "\n")
        plot(trace - ref_trace)

Again, you may need to modify this a little bit such as adding code to make a new `figure()`. Also notice in the above example how I reduced the number of samples.


In [ ]:
%matplotlib notebook
import matplotlib.pylab as plt

plt.figure()
ref_trace = cap_pass_trace("\x00\n")[0:500]

for c in 'abcdefghijklmnopqrstuvwxyz0123456789': 
    trace = cap_pass_trace(c + "\n")[0:500]
    plt.plot(trace - ref_trace)

OK great - hopefully you now see one major "difference". It should look something like this:
    
![](spa_password_diffexample.png)    

What do do now? Let's make this thing automatically detect such a large difference. Some handy stuff to try out is the `np.sum()` and `np.abs()` function.

The first one will get absolute values:

```python
import numpy as np
np.abs([-1, -3, 1, -5, 6])

    Out[]: array([1, 3, 1, 5, 6])
```

The second one will add up all the numbers.

```python
import numpy as np    
np.sum([-1, -3, 1, -5, 6])

    Out[]: -2
```

Using just `np.sum()` means positive and negative differences will cancel each other out - so it's better to do something like `np.sum(np.abs(DIFF))` to get a good number indicating how "close" the match was.


In [ ]:
import numpy as np
ref_trace = cap_pass_trace( "\x00\n")

for c in 'abcdefghijklmnopqrstuvwxyz0123456789': 
    trace = cap_pass_trace(c + "\n")
    diff = np.sum(np.abs(trace - ref_trace))
    
    print("{:1} diff = {:2}".format(c, diff))

Now the easy part - modify your above code to automatically print the correct password character. This should be done with a comparison of the `diff` variable - based on the printed characters, you should see one that is 'higher' than the others. Set a threshold somewhere reasonable (say I might use `25.0` based on one run).

## Running a Full Attack

Finally - let's finish this off. Rather than attacking a single character, we need to attack each character in sequence.

If you go back to the plotting of differences, you can try using the correct first character & wrong second character. The basic idea is exactly the same as before, but now we loop through 5 times, and just build up the password based on brute-forcing each character.

Take a look at the following for the basic pseudo-code:

    guessed_pw = "" #Store guessed password so far
    
    do a loop 5 times (max password size):
        
        ref_trace = capture power trace(guessed_pw + "\x00\n")
        
        for CHARACTER in LIST_OF_VALID_CHARACTERS:
            trace = capture power trace (guessed_pw + CHARACTER + newline)
            diff = SUM(ABS(trace - ref_trace))
            
            if diff > THRESHOLD:
                
                guessed_pwd += c
                print(guessed_pw)
                
                break


In [ ]:
guessed_pw = ""


for _ in range(0, 5):  

    ref_trace = cap_pass_trace(guessed_pw + "\x00\n")
    
    for c in 'abcdefghijklmnopqrstuvwxyz0123456789': 
        trace = cap_pass_trace(guessed_pw + c + "\n")
        diff = np.sum(np.abs(trace - ref_trace))

        if diff > 40.0:
            guessed_pw += c
            print(guessed_pw)
            break

You should get an output that looks like this:

    h
    h0
    h0p
    h0px
    h0px3

If so - 🥳🥳🥳🥳🥳🥳🥳🥳🥳🥳🥳🥳🥳 Congrats - you did it!!!!

If not - check some troubleshooting hints below. If you get really stuck, check the `SOLN` version (there is one for both with hardware and simulated).

## Troubleshooting - Always get 'h'

Some common problems you might run into - first, if you get an output which keeps guessing the first character:

    h
    hh
    hhh
    hhhh
    hhhhh

Check that when you run the `cap_pass_trace` inside the loop (checking the guessed password), are you updating the prefix of the password? For example, the old version of the code (guessing a single character) looked like this:

    trace = cap_pass_trace(c + "\n")

But that is always sending our first character only! So we need to send the "known good password so far". In the example code something like this:
   
    trace = cap_pass_trace(guessed_pw + c + "\n")

Where `guessed_pw` progressively grows with the known good start of the password.

#### Troubleshooting - Always get 'a'

This looks like it's always matching the first character:

    h
    ha
    haa
    haaa
    haaaa

Check that you update the `ref_trace` - if you re-use the original reference trace, you won't be looking at a reference where the first N characters are good, and the remaining characters are bad. An easy way to do this is again using the `guessed_pw` variable and appending a null + newline:

    trace = cap_pass_trace(guessed_pw + "\x00\n")